## With Tensorflow Keras

In [1]:
import numpy as np
import keras
import tensorflow as tf
from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler
from keras.datasets import cifar10
from keras.layers import Conv2D, MaxPool2D,Dropout, Dense, Input, concatenate, GlobalAveragePooling2D, AveragePooling2D,Flatten,  Average
import tensorflow_datasets as tfds

2024-08-06 06:46:09.141424: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-06 06:46:09.141531: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-06 06:46:09.251763: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Inception Module

In [2]:
class InceptionModule(keras.Model):
    def __init__(self, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj, name=None):
        super().__init__()
        self.conv_1x1 = Conv2D(filters_1x1, (1,1), padding="same", activation="relu")
        self.conv_3x3_red = Conv2D(filters_3x3_reduce, (3,3), padding="same", activation="relu")
        self.conv_3x3 = Conv2D(filters_3x3, (3,3), padding="same", activation="relu")
        self.conv_5x5_red = Conv2D(filters_5x5_reduce, (5,5), padding="same", activation="relu")
        self.conv_5x5 = Conv2D(filters_5x5, (5,5), padding="same", activation="relu")
        self.pool_proj_red = MaxPool2D((3,3), strides=(1,1), padding="same")
        self.pool = Conv2D(filters_pool_proj, (1,1), padding="same", activation="relu")
        self.name = name
    
    def call(self, X):
        conv_1 = self.conv_1x1(X)
        conv_3_red = self.conv_3x3_red(X)
        conv_3 = self.conv_3x3(conv_3_red)
        conv_5_red = self.conv_5x5_red(X)
        conv_5 = self.conv_5x5(conv_5_red)
        pool_red = self.pool_proj_red(X)
        pool = self.pool(pool_red)
        output = concatenate([conv_1, conv_3, conv_5, pool], axis=3, name = self.name)
        return output          

## Test InceptionModule

In [3]:
inception_module = InceptionModule(
    filters_1x1=64,
    filters_3x3_reduce=96,
    filters_3x3=128,
    filters_5x5_reduce=16,
    filters_5x5=32,
    filters_pool_proj=32,
    name="inception_block_1"
)

input_tensor = keras.Input(shape=(224, 224, 3))

output_tensor = inception_module(input_tensor)
print(output_tensor)

<KerasTensor shape=(None, 224, 224, 256), dtype=float32, sparse=False, name=keras_tensor_2>


## Google Net architecture

In [4]:
class GoogLeNet(keras.Model):
    def __init__(self, num_classes):
        super(GoogLeNet, self).__init__()
        
        
        self.conv_1_7x7 = Conv2D(64, (7,7),   padding="same", strides=(2,2), activation="relu")
        self.max_pool_1 = MaxPool2D((3,3),    padding="same", strides=(2,2))
        self.conv_2_1x1 = Conv2D(64, (1,1),   padding="same", strides=(1,1), activation="relu")
        self.conv_3_3x3 = Conv2D(192, (3,3),  padding="same", strides=(1,1), activation="relu")
        self.max_pool_2 = MaxPool2D((3,3),    padding="same", strides=(2,2))
        
        self.inception_1 = InceptionModule(filters_1x1=64, filters_3x3_reduce=96, filters_3x3=128,filters_5x5_reduce=16, filters_5x5=32,filters_pool_proj=32, name="inception_3a")
        self.inception_2 = InceptionModule(filters_1x1=128,filters_3x3_reduce=128, filters_3x3=192,filters_5x5_reduce=32,  filters_5x5=96,filters_pool_proj=64,  name="inception_3b")
        
        self.max_pool_3 = MaxPool2D((3,3), padding="same", strides = (2,2))
        
        self.inception_3 = InceptionModule(filters_1x1=192,filters_3x3_reduce=96, filters_3x3=208,filters_5x5_reduce=16,filters_5x5=48,filters_pool_proj=64,  name="inception_4a")
        
        self.flatten =  Flatten()
        
        # Output 1
        self.x1_avg = AveragePooling2D((5,5), strides=3)
        self.x1_conv = Conv2D(128, (1,1), padding="same", activation="relu")
        self.x1_dense1 = Dense(1024, activation="relu")
        self.x1_drop = Dropout(0.7)
        self.x1_out = Dense(num_classes, activation="softmax")
        
        self.inception_4 = InceptionModule(filters_1x1=160,filters_3x3_reduce=112, filters_3x3=224,filters_5x5_reduce=24,filters_5x5=64, filters_pool_proj=64,name="inception_4b")
        self.inception_5 = InceptionModule(filters_1x1=128,filters_3x3_reduce=128, filters_3x3=256,filters_5x5_reduce=24,filters_5x5=64, filters_pool_proj=64,name="inception_4c")
        self.inception_6 = InceptionModule(filters_1x1=112,filters_3x3_reduce=144, filters_3x3=288,filters_5x5_reduce=32,filters_5x5=64, filters_pool_proj=64,  name="inception_4d")
        
        self.x2_avg = AveragePooling2D((5,5), strides=3)
        self.x2_conv = Conv2D(128,(1,1), padding="same", activation="relu")
        self.x2_dense1 = Dense(1024, activation="relu")
        self.x2_drop = Dropout(0.7)
        self.x2_out = Dense(num_classes, activation="softmax")
        
        self.inception_7 = InceptionModule(filters_1x1=256,filters_3x3_reduce=160, filters_3x3=320,filters_5x5_reduce=32,filters_5x5=128,  filters_pool_proj=128,  name="inception_4e")
        
        self.max_pool_4 = MaxPool2D((3,3), padding="same", strides=(2,2))
        
        self.inception_8 = InceptionModule(filters_1x1=256,filters_3x3_reduce=160, filters_3x3=320,filters_5x5_reduce=32,filters_5x5=128,  filters_pool_proj=128,  name="inception_5a")
        self.inception_9 = InceptionModule(filters_1x1=384,filters_3x3_reduce=192, filters_3x3=384,filters_5x5_reduce=48,filters_5x5=128, filters_pool_proj=128, name="inception_5b")
        
        self.x3_avg = GlobalAveragePooling2D()
        self.x3_drop = Dropout(0.3)
        self.x3_out = Dense(num_classes, activation="softmax")
        
        self.avg_out = Average()
        
    def call(self, X):
            
        X = self.conv_1_7x7(X)
        X = self.max_pool_1(X)
        X = self.conv_2_1x1(X)
        X = self.conv_3_3x3(X)
        X = self.max_pool_2(X)
        
        X = self.inception_1(X)
        X = self.inception_2(X)
        X = self.max_pool_3(X)
        X = self.inception_3(X)
        
        X1 = self.x1_avg(X)
        X1 = self.x1_conv(X1)
        X1 = self.flatten(X1)
        X1 = self.x1_dense1(X1)
        X1 = self.x1_drop(X1)
        X1 = self.x1_out(X1)
        
        X = self.inception_4(X)
        X = self.inception_5(X)
        X = self.inception_6(X)
        
        X2 = self.x2_avg(X)
        X2 = self.x2_conv(X2)
        X2 = self.flatten(X2)
        X2 = self.x2_dense1(X2)
        X2 = self.x2_drop(X2)
        X2 = self.x2_out(X2)
                                 
        X = self.inception_7(X)
        X = self.inception_8(X)
        X = self.max_pool_4(X)
        X = self.inception_9(X)
        
        X = self.x3_avg(X)
        X = self.x3_drop(X)
        X = self.x3_out(X)
        out = self.avg_out([X, X1, X2])
        return   out

## Test GoogLeNet

In [5]:
google_net = GoogLeNet(10)

input_tensor = keras.Input(shape=(224, 224, 3))

output_tensor = google_net(input_tensor)
print(output_tensor)

<KerasTensor shape=(None, 10), dtype=float32, sparse=False, name=keras_tensor_14>


## Train with cats_vs_dogs dataset

In [6]:
dataset = tfds.load('cats_vs_dogs', split='train[:70%]', as_supervised=True)

train_dataset = dataset.take(int(len(dataset) * 0.8))
test_dataset = dataset.skip(int(len(dataset) * 0.8))


def preprocess(image, label):
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_dataset = train_dataset.map(preprocess).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(preprocess).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Corrupt JPEG data: 239 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 128 extraneous bytes before marker 0xd9
Corrupt JPEG data: 99 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1153 extraneous bytes before marker 0xd9
Corrupt JPEG data: 396 extraneous bytes before marker 0xd9
Corrupt JPEG data: 228 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9
Corrupt JPEG data: 252 extraneous bytes before marker 0xd9
Corrupt JPEG data: 2226 extraneous bytes before marker 0xd9
Corrupt JPEG data: 65 extraneous bytes before marker 0xd9


Shuffling /root/tensorflow_datasets/cats_vs_dogs/4.0.1.incomplete383XN5/cats_vs_dogs-train.tfrecord*...:   0%|…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [7]:
train_dataset = train_dataset.batch(batch_size=32)

In [8]:
num_classes = 2
input_shape = (224,224,3)

In [9]:
model = GoogLeNet(num_classes)

In [10]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(train_dataset, epochs=10)

Epoch 1/10


I0000 00:00:1722926940.330161     143 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1722926940.388659     143 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1722926940.389291     143 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1722926940.389902     143 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


408/408 ━━━━━━━━━━━━━━━━━━━━ 168s 236ms/step - accuracy: 0.5117 - loss: 0.6938
Epoch 2/10


W0000 00:00:1722927036.625683     145 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1722927036.631937     145 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


408/408 ━━━━━━━━━━━━━━━━━━━━ 65s 158ms/step - accuracy: 0.5075 - loss: 0.6931
Epoch 3/10
408/408 ━━━━━━━━━━━━━━━━━━━━ 64s 157ms/step - accuracy: 0.5076 - loss: 0.6931
Epoch 4/10
408/408 ━━━━━━━━━━━━━━━━━━━━ 64s 157ms/step - accuracy: 0.5078 - loss: 0.6931
Epoch 5/10
283/408 ━━━━━━━━━━━━━━━━━━━━ 19s 157ms/step - accuracy: 0.5089 - loss: 0.6930